In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import itertools # Importando itertools para generar combinaciones de columnas
# Importando la función seasonal_decompose para la descomposición de series temporales
from statsmodels.tsa.seasonal import seasonal_decompose
import matplotlib.patches as mpatches
import payments_manager as pm
#pm.help()
#pm.reset()
pm.init() #debug=True)
cr_cp = pm.df('cr_cp')
fe_cp = pm.df('fe_cp')
#cr_cp.info()
#fe_cp.info()

df_jo = pm.df('df_jo')
#df_jo.info()
df_jo = pm.sort("df_jo", ["id_cr"]).reset_index()
df_jo = df_jo.drop(columns=['index'])
#df_jo = df_jo.drop(columns=['Mes_created_at'])
df_jo_cp = df_jo.copy()

In [2]:
df_jo.info()
display(df_jo)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32094 entries, 0 to 32093
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   id_cr                       32094 non-null  int64         
 1   amount                      32094 non-null  float64       
 2   stat_cr                     32094 non-null  object        
 3   created_at                  32094 non-null  datetime64[ns]
 4   user_id                     32094 non-null  int64         
 5   active                      32094 non-null  int64         
 6   moderated_at                21759 non-null  datetime64[ns]
 7   reimbursement_date          32094 non-null  datetime64[ns]
 8   cash_request_received_date  24149 non-null  datetime64[ns]
 9   money_back_date             23917 non-null  datetime64[ns]
 10  transfer_type               32094 non-null  object        
 11  send_at                     22678 non-null  datetime64

,id_cr,amount,stat_cr,created_at,user_id,active,moderated_at,reimbursement_date,cash_request_received_date,money_back_date,...,Mes_created_at,type,stat_fe,category,total_amount,paid_at,from_date,to_date,charge_moment,fee
0,3,1.0,canceled,2019-11-19 13:57:53.511561,47,1,2019-11-20 12:16:50.000000,2019-12-05 23:00:00.000000,NaT,NaT,...,2019-11,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
1,4,100.0,money_back,2019-12-09 14:47:35.190714,99001309,0,2019-12-09 14:58:28.000000,2019-12-16 23:00:00.000000,NaT,NaT,...,2019-12,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
2,5,100.0,rejected,2019-12-10 19:05:21.596873,804,1,2019-12-11 16:47:42.405646,2020-01-09 19:05:21.596363,NaT,NaT,...,2019-12,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
3,6,100.0,direct_debit_rejected,2019-12-10 19:05:48.921042,812,1,2019-12-11 09:40:33.000000,2020-02-05 23:00:00.000000,2019-12-11,NaT,...,2019-12,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
4,7,100.0,rejected,2019-12-10 19:13:35.825460,191,1,2019-12-11 09:46:59.777728,2020-01-09 19:13:35.825041,NaT,NaT,...,2019-12,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32089,27006,50.0,money_back,2020-11-01 23:07:55.231487,55916,1,NaT,2020-11-12 22:00:00.000000,2020-11-02,2020-11-12 19:39:20.241980,...,2020-11,instant_payment,accepted,NaN,5.0,2020-11-12 19:39:20.299302,NaT,NaT,after,5.0
32090,27007,100.0,money_back,2020-11-01 23:11:37.788849,39339,1,NaT,2020-11-29 11:00:00.000000,2020-11-02,2020-11-29 19:27:43.055298,...,2020-11,instant_payment,accepted,NaN,5.0,2020-11-29 19:27:43.101686,NaT,NaT,after,5.0
32091,27008,50.0,rejected,2020-11-01 23:13:19.286745,24505,1,2020-11-02 09:06:37.162733,2020-11-24 23:51:00.000000,NaT,NaT,...,2020-11,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaN,NaN
32092,27009,60.0,money_back,2020-11-01 23:15:28.102894,100781,1,NaT,2020-12-04 11:00:00.000000,NaT,2020-12-02 14:17:37.433540,...,2020-11,instant_payment,accepted,NaN,5.0,2020-12-02 14:17:37.499800,NaT,NaT,after,5.0


In [11]:
df_jall = pm.df('df_jall')
df_jall.info()
display(df_jall)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32094 entries, 0 to 32093
Data columns (total 31 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_cr               32094 non-null  int64         
 1   amount              32094 non-null  float64       
 2   stat_cr             32094 non-null  object        
 3   created_at          32094 non-null  datetime64[ns]
 4   updated_at_x        32094 non-null  datetime64[ns]
 5   user_id             32094 non-null  int64         
 6   active              32094 non-null  int64         
 7   moderated_at        21759 non-null  datetime64[ns]
 8   reimbursement_date  32094 non-null  datetime64[ns]
 9   cr_received_date    24149 non-null  datetime64[ns]
 10  money_back_date     23917 non-null  datetime64[ns]
 11  transfer_type       32094 non-null  object        
 12  send_at             22678 non-null  datetime64[ns]
 13  recovery_status     7200 non-null   object    

,id_cr,amount,stat_cr,created_at,updated_at_x,user_id,active,moderated_at,reimbursement_date,cr_received_date,...,category,total_amount,reason,created_at_y,updated_at_y,paid_at,from_date,to_date,charge_moment,fee
0,5,100.0,rejected,2019-12-10 19:05:21.596873,2019-12-11 16:47:42.407830,804,1,2019-12-11 16:47:42.405646,2020-01-09 19:05:21.596363,NaT,...,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN
1,70,100.0,rejected,2019-12-10 19:50:12.347780,2019-12-11 14:24:22.900054,231,1,2019-12-11 14:24:22.897988,2020-01-09 19:50:12.347780,NaT,...,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN
2,7,100.0,rejected,2019-12-10 19:13:35.825460,2019-12-11 09:46:59.779773,191,1,2019-12-11 09:46:59.777728,2020-01-09 19:13:35.825041,NaT,...,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN
3,10,99.0,rejected,2019-12-10 19:16:10.880172,2019-12-18 14:26:18.136163,761,1,2019-12-18 14:26:18.128407,2020-01-09 19:16:10.879606,NaT,...,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN
4,1594,100.0,rejected,2020-05-06 09:59:38.877376,2020-05-07 09:21:55.340080,7686,1,2020-05-07 09:21:55.320193,2020-06-05 22:00:00.000000,NaT,...,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32089,22357,100.0,money_back,2020-10-20 07:58:04.006937,2021-02-05 12:19:30.656816,82122,1,NaT,2021-02-05 11:00:00.000000,2020-10-21,...,NaN,5.0,Instant Payment Cash Request 22357,2020-10-20 07:58:19.637461,2020-10-20 07:58:19.637491,2021-02-05 12:19:30.685779,NaT,NaT,after,5.0
32090,20256,100.0,money_back,2020-10-10 05:40:55.700422,2021-02-05 13:14:19.707627,64517,1,NaT,2021-02-05 11:00:00.000000,2020-10-12,...,NaN,5.0,Postpone Cash Request 20256,2020-10-30 17:08:16.906449,2020-10-30 17:08:21.967966,2020-10-30 17:08:21.416796,2020-11-06 11:00:00,2020-12-07 11:00:00,before,5.0
32091,20256,100.0,money_back,2020-10-10 05:40:55.700422,2021-02-05 13:14:19.707627,64517,1,NaT,2021-02-05 11:00:00.000000,2020-10-12,...,NaN,5.0,Instant Payment Cash Request 20256,2020-10-10 05:42:11.679401,2020-10-13 14:25:03.460352,2021-02-05 13:14:19.731397,NaT,NaT,after,5.0
32092,19886,100.0,direct_debit_sent,2020-10-08 14:16:52.155661,2021-01-05 15:45:52.645536,44867,1,NaT,2021-02-05 11:00:00.000000,2020-10-10,...,NaN,5.0,Postpone Cash Request 19886,2020-10-10 21:22:00.083739,2020-10-13 14:25:18.501716,2020-10-10 21:22:04.456828,2020-11-06 22:00:00,2020-12-06 11:00:00,before,5.0
